### Format

    Input
    """
    질문: <question>
    풀이: <reasoning>
    정답: <answer>

    질문: <question>
    풀이: 
    """
    
    
    Output
    """
    <reasoning>
    정답 : <answer>
    """
    

## 각각의 데이터는 같음
    - /userhomes/philhoon/kt-ai-challenge/result
    # kor-gsm8k-one_shot_cot_finetuning-test.jsonl
    # kor-gsm8k-peft2-test.jsonl

    # kor-gsm8k-one_shot_cot_finetuning-train.csv
    # kor-gsm8k-peft2-train.csv

In [4]:
import json
import requests
from src import utils
import time
import os
import csv
import re

In [5]:
# PROMPT1 = "질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? \
# 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. \
# 정답: 72 \\n"


In [28]:
# 질문: 항아리 안에 몇 개의 젤리빈이 있습니다. 젤리빈의 3/4은 빨간색이며, 빨간색 젤리빈의 1/4은 코코넛 맛입니다. 코코넛 맛 젤리빈이 750개이면, 항아리 안에는 몇 개의 젤리빈이 있나요? 풀이: '
# 젤리콩이 750*4=3000개가 있습니다. 그리고 병 안에는 3000/3*4=4000개의 젤리콩이 있습니다. 정답: 4000
PROMPT2 = "질문: 항아리 안에 몇 개의 젤리빈이 있습니다. 젤리빈의 3/4은 빨간색이며, 빨간색 젤리빈의 1/4은 코코넛 맛입니다. 코코넛 맛 젤리빈이 750개이면, 항아리 안에는 몇 개의 젤리빈이 있나요? \
풀이: 젤리콩이 750*4=3000개가 있습니다. 그리고 병 안에는 3000/3*4=4000개의 젤리콩이 있습니다. \
정답: 4000 \\n"


In [6]:
class model_api():
    def __init__(self, url, headers):
        self.url = url
        self.headers = headers
        
    def generate(self, body):
        
        try:
            response = requests.post(self.url, data=body, headers=self.headers, verify=False)
        
        except:
            print("retrying due to an error......")
            time.sleep(20)
            return self.generate(body)
        
        return response
        
        

In [7]:
api_path = '/userhomes/philhoon/kt-ai-challenge/api-peft2.json'
api = utils.read_json_file(api_path)

In [8]:
BODY_TEMP = api['body']

In [9]:
model = model_api(api['url'], api['headers'])

In [10]:
# input_file = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl'
input_file = '/userhomes/philhoon/kt-ai-challenge/result/kor-gsm8k-peft2-test.jsonl'

In [11]:
data = utils.read_jsonlines(input_file)

In [12]:
len(data)

1293

In [15]:
# os.makedirs(output_path, exist_ok=True)
output_path = '/userhomes/philhoon/kt-ai-challenge/result'
output_file_name = 'peft2-result-ko-gsm8k-test.jsonl'
output_file = output_path + '/' + output_file_name
print(f'output_file : {output_file}')
# jsonl_writer = utils.JSONLWriter(output_file)

output_file : /userhomes/philhoon/kt-ai-challenge/result/peft2-result-ko-gsm8k-test.jsonl


In [16]:
# def get_gsm8k_zs(que):
#     prompt = f'질문 : {que} \n정답 : '
#     return prompt

In [17]:
# def get_gsm8k_build_data_with_reason(kor_que, kor_ans, ans_s):
#     prompt = f"질문: {kor_que}"
#     output = f"풀이: {kor_ans} 정답: {ans_s}"
#     return prompt, output

In [18]:
def get_body(prompt, body_temp):
    body_temp["utterance"] = prompt
    return json.dumps(body_temp)

In [19]:
# from pprint import pprint

In [20]:
def remove_line_slit(context):
    context = re.sub(r"\n", " ", context)
    return context

In [30]:
def get_gsm8k_build_data_with_one_shot_cot_finetuning(kor_que, kor_ans, ans_s, PROMPT1):
    PROMPT1 
    prompt = f"질문: {kor_que} 풀이: "
    output = f"{kor_ans} 정답: {ans_s}"
    
    prompt = PROMPT1 + prompt
    return prompt, output

### GSM8K-TEST
    26 out of 1319 : contains the answer in the question due to translation
### GSM8K-TRAIN
    163 out of 7473 : contains the answer in the question due to translation

In [21]:
from pprint import pprint

In [35]:
cnt = 0
cnt_fix = 0
split_cnt = 0
input_lst = []
output_lst = []

for ins in data:
#     prompt = ins['kor_question_reason']

    kor_que = ins['kor_question']
    kor_ans = ins['kor_answer']
    ans_s = ins['answer_s']
    kor_ans = ins['kor_answer']
    
    prompt, out_ = get_gsm8k_build_data_with_one_shot_cot_finetuning(kor_que, kor_ans, ans_s, PROMPT2)


    body = get_body(prompt, BODY_TEMP)
    response = model.generate(body)
    res_context = response.json()['data'][0]['result'][0]
    
    print(f'prompt : ')
    print(prompt)
    print(f'res_context : ')
    print(res_context)
    print('----')
    
#     break
    # checking whether there is an answer 
#     test = kor_que.split('\n')
    
#     if len(test[0]) != len(kor_que):
#         split_cnt += 1
#         continue
    
#     print(f'kor_que')
#     print(f'{kor_que}')
#     print(f'kor_ans')
#     print(f'{kor_ans}')
#     print(f'ans_s')
#     print(f'{ans_s}')
    
#     ins['peft1_infer'] = res_context
#     jsonl_writer.write_json_line(ins)
    
    if cnt == 20:
        break
    else:
        cnt += 1

/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문: 항아리 안에 몇 개의 젤리빈이 있습니다. 젤리빈의 3/4은 빨간색이며, 빨간색 젤리빈의 1/4은 코코넛 맛입니다. 코코넛 맛 젤리빈이 750개이면, 항아리 안에는 몇 개의 젤리빈이 있나요? 풀이: 젤리콩이 750*4=3000개가 있습니다. 그리고 병 안에는 3000/3*4=4000개의 젤리콩이 있습니다. 정답: 4000 \n질문: 한 벌의 로브를 만드는 데 파란색 섬유 2 덩이와 그 절반만큼의 흰색 섬유가 필요합니다. 총 몇 덩이의 섬유가 필요한가요? 풀이: 
res_context : 
그리고 병 안에는 2000/3*4=12000개의 섬유가 있습니다. 정답: 12000 \n질문: 한 벌의 로브를 만드는 데 파란색 섬유 2 덩이와 그 절반만큼의 흰색 섬유가 필요합니다.
----


/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문: 항아리 안에 몇 개의 젤리빈이 있습니다. 젤리빈의 3/4은 빨간색이며, 빨간색 젤리빈의 1/4은 코코넛 맛입니다. 코코넛 맛 젤리빈이 750개이면, 항아리 안에는 몇 개의 젤리빈이 있나요? 풀이: 젤리콩이 750*4=3000개가 있습니다. 그리고 병 안에는 3000/3*4=4000개의 젤리콩이 있습니다. 정답: 4000 \n질문: 조쉬는 집을 뒤집어 판매해보기로 결정합니다. 그는 $80,000에 집을 구매하고 $50,000을 수리합니다. 이로 인해 집의 가치가 150% 증가합니다. 그가 얼마의 이익을 얻었습니까? 풀이: 
res_context : 
그리고 $80과 $50의 차액을 가지고 있습니다. 조쉬는 $80과 $50의 차액을 가지고 있으므로 총 이익은 $150입니다. 정답: 150 \n질문: 조쉬는 $80을 가지고 있고
----


/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

prompt : 
질문: 항아리 안에 몇 개의 젤리빈이 있습니다. 젤리빈의 3/4은 빨간색이며, 빨간색 젤리빈의 1/4은 코코넛 맛입니다. 코코넛 맛 젤리빈이 750개이면, 항아리 안에는 몇 개의 젤리빈이 있나요? 풀이: 젤리콩이 750*4=3000개가 있습니다. 그리고 병 안에는 3000/3*4=4000개의 젤리콩이 있습니다. 정답: 4000 \n질문: 마리사는 12마일 길을 하이킹하고 있습니다. 그녀는 처음 4마일을 걷는 데 1시간이 걸렸으며, 다음 2마일을 걷는 데 또 다른 1시간이 걸렸습니다. 그녀가 평균 속도를 시간당 4마일로 유지하려면, 남은 거리를 걷기 위해 어떤 속도(마일당 시간)로 걷어야 합니까? 풀이: 
res_context : 
그리고 병 안에는 3000/3*4=4000개의 젤리콩이 있습니다. 젤리콩의 3/4은 빨간색이고 4000은 초록색입니다. 병 안에는 7500개의 젤리빈이 있습니다. 정답: 4000 마리사가 12마일 길을 하이킹하고 있습니다. 그녀는 처음 4마일을 걷는 데 1시간이 걸렸으며, 다음 2마일을 걷는
----


/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문: 항아리 안에 몇 개의 젤리빈이 있습니다. 젤리빈의 3/4은 빨간색이며, 빨간색 젤리빈의 1/4은 코코넛 맛입니다. 코코넛 맛 젤리빈이 750개이면, 항아리 안에는 몇 개의 젤리빈이 있나요? 풀이: 젤리콩이 750*4=3000개가 있습니다. 그리고 병 안에는 3000/3*4=4000개의 젤리콩이 있습니다. 정답: 4000 \n질문: 저는 2/3이 물인 오렌지 음료를 10리터 가지고 있고, 3/5가 물인 파인애플 음료 15리터에 추가하려고 합니다. 그러나 따르는 도중에 오렌지 음료 1리터를 쏟았습니다. 남은 24리터에는 얼마나 많은 물이 있나요? 풀이: 
res_context : 
그리고 병 안에는 3000/3*4=6000개의 젤리콩이 있습니다. 정답: 6000 \n질문: 한 항아리의 물은 2/3이 물이고 3/5가 물입니다. 한 항아리의 물을 2/3
----


/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문: 항아리 안에 몇 개의 젤리빈이 있습니다. 젤리빈의 3/4은 빨간색이며, 빨간색 젤리빈의 1/4은 코코넛 맛입니다. 코코넛 맛 젤리빈이 750개이면, 항아리 안에는 몇 개의 젤리빈이 있나요? 풀이: 젤리콩이 750*4=3000개가 있습니다. 그리고 병 안에는 3000/3*4=4000개의 젤리콩이 있습니다. 정답: 4000 \n질문: 레이몬드와 사만다는 사촌입니다. 레이몬드는 사만다보다 6년 먼저 태어났습니다. 레이몬드는 23살 때 아들을 가졌습니다. 사만다가 지금 31살이라면, 레이몬드의 아들이 태어난 지 몇 년이 지났을까요? 풀이: 
res_context : 
그리고 병 안에는 3000/3*4=1000개의 젤리콩이 있습니다. 정답: 1000 \n질문: 레이몬드는 사만다보다 6년 먼저 태어났습니다. 사만다는 레이몬드보다 23살 더 많습니다.
----


In [ ]:
len(data) - split_cnt

In [ ]:
len(input_lst)

In [ ]:
len(output_lst)

In [ ]:
def save_csv_file(file_path, sentence_a_list, sentence_b_list):
    with open(file_path, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        for sentence_a, sentence_b in zip(sentence_a_list, sentence_b_list):
            writer.writerow([sentence_a, sentence_b])

In [ ]:
output_path = '/userhomes/philhoon/kt-ai-challenge/result'

In [ ]:
output_file = output_path + '/kor-gsm8k-reason-train.csv'

In [ ]:
output_file

In [ ]:
save_csv_file(output_file, input_lst, output_lst)

In [ ]:
input_lst[1]

In [ ]:
output_lst[1]